In [2]:


!pip install tensorflow mediapipe

import os
import numpy as np
import pickle
from tqdm import tqdm
import tensorflow as tf
import mediapipe as mp
import cv2



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 57.5 MB/s eta 0:00:00


In [4]:
# GPU setup with minimal memory growth and precision
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        try:
            tf.config.experimental.set_memory_growth(device, True)
            tf.config.set_logical_device_configuration(
                device, [tf.config.LogicalDeviceConfiguration(memory_limit=15000)]
            )
            print("GPU configured with a memory limit of 15000 MB.")
        except Exception as e:
            print(f"Error configuring GPU: {e}")
else:
    print("No GPU detected, running on CPU.")

try:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled for speedup.")
except ValueError:
    print("Mixed precision not supported, running with default precision.")

# Function to save progress
def save_progress(filepath, data):
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)

# Function to load progress
def load_progress(filepath):
    if os.path.exists(filepath):
        try:
            with open(filepath, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            print(f"Error loading checkpoint {filepath}: {e}")
    return []

# Function to extract landmarks from images
def extract_landmarks_from_images(image_paths):
    mp_face_mesh = mp.solutions.face_mesh
    landmarks_list = []

    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
        for image_path in tqdm(image_paths, desc="Processing Images"):
            image = cv2.imread(image_path)
            if image is None:
                print(f"Failed to read image: {image_path}")
                landmarks_list.append([(0, 0, 0)] * 468)
                continue

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(image_rgb)
            if results.multi_face_landmarks:
                face_landmarks = results.multi_face_landmarks[0]
                landmarks = [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark]
                landmarks_list.append(landmarks)
            else:
                landmarks_list.append([(0, 0, 0)] * 468)

    return landmarks_list

# Function to process images and save landmarks with checkpoints
def extract_landmark_features_with_checkpoint(image_paths, checkpoint_path):
    saved_progress = load_progress(checkpoint_path)
    processed_images = {entry['image_path'] for entry in saved_progress}
    remaining_images = [img for img in image_paths if img not in processed_images]

    print(f"Total images: {len(image_paths)}, Remaining images: {len(remaining_images)}")

    for image_path in tqdm(remaining_images, desc="Extracting Landmark Features"):
        try:
            landmarks = extract_landmarks_from_images([image_path])[0]
            saved_progress.append({'image_path': image_path, 'landmarks': landmarks})
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
        save_progress(checkpoint_path, saved_progress)

    return saved_progress

if __name__ == "__main__":
    # Define paths and directories
    base_path = 'drive/MyDrive/SP_cup/features/fake/'
    os.makedirs(base_path, exist_ok=True)

    real_images_path = 'drive/MyDrive/SP_cup/fake/fake-2/'

    real_checkpoint = os.path.join(base_path, 'landmarks_fake1.pkl')

    # Collect image paths efficiently
    def get_image_paths(folder_path):
        return [
            os.path.join(root, file)
            for root, _, files in os.walk(folder_path)
            for file in files if file.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]

    real_image_paths = get_image_paths(real_images_path)

    # Extract landmark features
    print("Processing real images...")
    landmark_features_real = extract_landmark_features_with_checkpoint(real_image_paths, real_checkpoint)

    print("Landmark feature extraction completed.")


Error configuring GPU: Cannot set memory growth on device when virtual devices configured
Mixed precision enabled for speedup.
Processing real images...
Total images: 7000, Remaining images: 7000


Streaming output truncated to the last 5000 lines.
Extracting Landmark Features: 100%|██████████| 7000/7000 [1:54:48<00:00,  1.02it/s]

Landmark feature extraction completed.
